## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-09-18-56-18 +0000,nypost,Drone packed with crab-boil fixin’s — even jar...,https://nypost.com/2025/12/09/us-news/drone-pa...
1,2025-12-09-18-55-26 +0000,bbc,Judge rules Ghislaine Maxwell grand jury recor...,https://www.bbc.com/news/articles/cn8e2kvzen7o...
2,2025-12-09-18-54-38 +0000,bbc,Labour and Plaid Cymru agree Welsh budget deal,https://www.bbc.com/news/articles/cp8zjpeq3z2o...
3,2025-12-09-18-52-53 +0000,nypost,EU probes Google for possible ‘anticompetitive...,https://nypost.com/2025/12/09/business/europea...
4,2025-12-09-18-50-47 +0000,bbc,Budget could knock 0.5% off inflation next yea...,https://www.bbc.com/news/articles/c898djgn2n2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,53
75,new,21
147,up,15
390,china,14
367,ukraine,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
242,2025-12-09-01-07-00 +0000,wsj,President Trump backed away from a vow to rele...,https://www.wsj.com/politics/national-security...,114
154,2025-12-09-11-56-07 +0000,nypost,"Trump warns Ukraine is ‘losing’ Russia war, ca...",https://nypost.com/2025/12/09/us-news/trump-sa...,112
248,2025-12-09-00-29-40 +0000,nypost,Trump gives green light to Nvidia to ship powe...,https://nypost.com/2025/12/08/business/trump-g...,109
155,2025-12-09-11-54-00 +0000,wsj,President Trump now has enormous power over th...,https://www.wsj.com/politics/policy/in-warner-...,106
232,2025-12-09-02-13-58 +0000,bbc,Trump gives Nvidia green light to sell advance...,https://www.bbc.com/news/articles/ckg9q635q6po...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
242,114,2025-12-09-01-07-00 +0000,wsj,President Trump backed away from a vow to rele...,https://www.wsj.com/politics/national-security...
129,90,2025-12-09-13-30-50 +0000,bbc,Ukraine prepares new peace plan as Zelensky ru...,https://www.bbc.com/news/articles/cg7vdd115vjo...
251,72,2025-12-09-00-18-55 +0000,wapo,Paramount launches hostile bid for Warner Bros...,https://www.washingtonpost.com/business/2025/1...
248,56,2025-12-09-00-29-40 +0000,nypost,Trump gives green light to Nvidia to ship powe...,https://nypost.com/2025/12/08/business/trump-g...
77,50,2025-12-09-16-11-00 +0000,wsj,"Bruce Blakeman, a Long Island Republican who t...",https://www.wsj.com/politics/elections/long-is...
59,41,2025-12-09-17-00-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
306,41,2025-12-08-21-42-03 +0000,nypost,CBS News taps ABC News journalist Matt Gutman ...,https://nypost.com/2025/12/08/media/cbs-news-t...
318,37,2025-12-08-21-09-00 +0000,wsj,Supreme Court Appears Ready to Give President ...,https://www.wsj.com/politics/policy/supreme-co...
290,37,2025-12-08-22-26-00 +0000,wsj,The Trump administration announced $12 billion...,https://www.wsj.com/politics/policy/trump-to-u...
110,35,2025-12-09-14-32-52 +0000,nypost,Arkansas mom alerted politician that she feare...,https://nypost.com/2025/12/09/us-news/arkansas...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
